In [26]:
import json
import pandas as pd
import random
import numpy as np

clamp = 10
kerf = 10

# len, qty, g1, pitch


def get_g2(length, g1, pitch):
    return (length - g1) % pitch

def get_possible_quantity(long_len, long_g1, short_len, short_g1, pitch):
    possible_quantity = 0
    loss_at_cut = [0]
    len_at_point = [long_len]
    long_g2 = get_g2(long_len, long_g1, pitch)
    while (
        (short_g1 + kerf <= long_g1) & ((long_len - long_g1) >= (short_len - short_g1))
    ) | ((long_len - long_g1) > short_len):
        cut = 1
        front_loss = 0
        if long_g1 == short_g1:
            cut = 0
        elif short_g1 < long_g1:
            front_loss = long_g1 - short_g1
        else:
            front_loss = long_g1 + (pitch - short_g1)

        short_g2 = get_g2(short_len, short_g1, pitch)
        long_g1 = pitch - (short_g2 + kerf)
        possible_quantity += 1
        long_len -= short_len + front_loss + kerf
        if long_len < 0:
            break
        len_at_point.append(long_len)
        loss_at_cut.append(front_loss + kerf * cut)
    return possible_quantity, len_at_point, loss_at_cut, long_g1


In [27]:
get_possible_quantity(3240, 30, 640, 30, 60)

(4, [3240, 2590, 1930, 1270, 610], [0, 0, 20, 20, 20], 30)

In [40]:
# while dem_arr['qty'].sum() > 0:
# stock_arr = [
#     [3240, 100, 30, 60]
# ]
# stock = pd.DataFrame(stock_arr, columns=['product_length','qty','product_g1', 'product_pitch'])    
dem_arr =[
        [640, 12, 30, 60],
        [400, 20, 30, 60],
        [280, 34, 30, 60],
    ]
demand = pd.DataFrame(dem_arr, columns=['product_length','qty','product_g1', 'product_pitch'])    
source_len = 3240
source_g1 = 30
longest_index = 0
pitch = 60
rail_record = {}
utilized = 0

# while demand['qty'].sum() > 0:
longest = demand.loc[0]
longest_quantity = longest.qty
max_in_source_longest_qty, leng_at_cut, _, drop_g1 = get_possible_quantity(source_len, source_g1, longest.product_length, longest.product_g1, pitch)
if longest.qty >= max_in_source_longest_qty:
    source_used_qty = longest.qty // max_in_source_longest_qty
    longeset_allocated_qty = source_used_qty  * max_in_source_longest_qty
    # longest_remainder_quantity = longest.qty % max_in_source_longest_qty
    drop1_len = leng_at_cut[max_in_source_longest_qty]
else:
    drop1_len = leng_at_cut[longest_quantity]
    longeset_allocated_qty = longest_quantity
utilized += longeset_allocated_qty * longest.product_length
pool_df = demand.iloc[1:]
# best_index = 



        

In [48]:
demand

,product_length,qty,product_g1,product_pitch
0,640,12,30,60
1,400,20,30,60
2,280,34,30,60


In [61]:
import pulp

# Initial data
source_rail_length = 3240
demand_lengths = [640, 400, 280]
demand_quantities = [12, 20, 34]

# Create the problem
prob = pulp.LpProblem("RailCutting", pulp.LpMinimize)

# Variables
x_vars = pulp.LpVariable.dicts("x", range(len(demand_lengths)), lowBound=0, cat='Integer')
y_vars = pulp.LpVariable.dicts("y", ((i, j) for i in range(len(demand_lengths)) for j in range(int(source_rail_length/demand_lengths[i])+1)), lowBound=0, cat='Binary')

# Objective function
prob += pulp.lpSum(x_vars[i] for i in range(len(demand_lengths)))

# Constraints
for i in range(len(demand_lengths)):
    prob += pulp.lpSum(y_vars[(i, j)] * demand_lengths[i] for j in range(int(source_rail_length/demand_lengths[i])+1)) <= source_rail_length * x_vars[i]
    prob += pulp.lpSum(y_vars[(i, j)] for j in range(int(source_rail_length/demand_lengths[i])+1)) == demand_quantities[i]

# Solve the problem
prob.solve()

# Print results
print(f"Minimum number of source rails needed: {pulp.value(prob.objective)}")

Minimum number of source rails needed: 5.4074074


In [62]:
m = []
for i in range(0, 3):
    m.append(demand_lengths[i] * demand_quantities[i])
sum(m)

25200

In [86]:
from typing import List, Tuple
kerf = 10
def rail_allocation_blank(source_rail_length: int, source_rail_qty: int, final_products: List[Tuple[int, int]]):
    # Sort final products in descending order by length
    final_products.sort(key=lambda x: x[0], reverse=True)

    # Initialize source rails as a list with all rails empty
    source_rails = [source_rail_length] * source_rail_qty
    combination = [[] for i in range(source_rail_qty)]    
    # For each type of final product
    for length, qty in final_products:
        # For each product of this type
        for _ in range(qty):
            # Find the first source rail where this product can fit
            for i, available_length in enumerate(source_rails):
                if available_length >= length + kerf:
                    # If it fits, place the product on this rail and break the inner loop
                    source_rails[i] -= length + kerf
                    combination[i].append(length)
                    break
            else:
                # If no fitting rail found, return error
                return "Error: Not enough source rails"
    # Count the number of source rails used
    source_rails_used = len([rail for rail in source_rails if rail != source_rail_length])
    return source_rails_used, source_rails, combination

# Set source rail length and quantity
source_rail_length = 3040
source_rail_qty = 15

# Set final product lengths and quantities
final_products = [(640, 12), (400, 20), (280, 34)]  # (length, qty)
# final_products = [(1540, 1), (790, 26), (400, 4), (242, 13)]  # (length, qty)

# Calculate and print the minimum number of source rails used
a, b, c = rail_allocation_blank(source_rail_length, source_rail_qty, final_products)
a, c

(9,
 [[640, 640, 640, 640, 400],
  [640, 640, 640, 640, 400],
  [640, 640, 640, 640, 400],
  [400, 400, 400, 400, 400, 400, 400],
  [400, 400, 400, 400, 400, 400, 400],
  [400, 400, 400, 280, 280, 280, 280, 280, 280],
  [280, 280, 280, 280, 280, 280, 280, 280, 280, 280],
  [280, 280, 280, 280, 280, 280, 280, 280, 280, 280],
  [280, 280, 280, 280, 280, 280, 280, 280],
  [],
  [],
  [],
  [],
  [],
  []])

In [213]:
import time

def get_g2(length, g1, pitch):
    return (length - g1) % pitch

fit_memo = {}

def check_fit(available, short_len, short_g1, pitch):        
    long_len, long_g1 = available    
    key = f"{long_len} {long_g1} {short_len} {short_g1} {pitch}"
    # print(key)
    if key in fit_memo.keys():
        return fit_memo[key]        
    if (
        (short_g1 + kerf <= long_g1) & ((long_len - long_g1) >= (short_len - short_g1))
    ) | ((long_len - long_g1) > short_len):                
        if long_g1 == short_g1:            
            front_loss = 0
        elif short_g1 < long_g1:
            front_loss = long_g1 - short_g1
        else:
            front_loss = long_g1 + (pitch - short_g1)
        short_g2 = get_g2(short_len, short_g1, pitch)
        long_g1 = pitch - (short_g2 + kerf)
        long_len -= short_len + front_loss + kerf        
        fit_memo[key] = [True, long_len, long_g1]        
        return True, long_len, long_g1
    else:
        fit_memo[key] = [False, 0, 0]
        return False, _, _
                
def initial_rail_allocation(source, source_pitch: int, final_products: List[Tuple[int, int, int]]):   
    final_products.sort(key=lambda x: x[0], reverse=True)   
    source_rails = []
    combination = []    
    for source in source.itertuples():        
        source_rails += [[source.length, source.g1] for i in range(source.qty)]
        combination += [[source.length] for i in range(source.qty)]
    # print(source_rails)
    
    for length, qty, g1 in final_products:           
        for _ in range(qty):    
            for i, available_length in enumerate(source_rails):
                does_fit, long_len, long_g1 = check_fit(available_length, length, g1, source_pitch)
                if does_fit:    
                    source_rails[i] = [long_len, long_g1]
                    combination[i].append(length)
                    break
            else:    
                return "Error: Not enough source rails", ''
    source_sum, source_left, used_sum, counter = 0, 0, 0, 0    
    for s, c in zip(source_rails, combination):        
        if s[0] == c[0]:
            break
        source_left += s[0] if s[0] > 0 else 0
        source_sum += c[0]        
        used_sum += sum(c[1:])
        counter += 1
    # source_rails_used = len([s for s, c in zip(source_rails, combination) if s[0] != c[0]])    
    eff = used_sum / source_sum    
    return source_rails[:counter], combination[:counter], eff, source_left

source_pitch = 60
final_products = [(640, 12, 20), (400, 20, 20), (280, 34, 10)]  # (length, qty, g1)
source_df = pd.DataFrame([[7000, 2, 20], [3000, 10, 30]], columns=['length', 'qty', 'g1'])
source_rails, combination, eff, source_left = initial_rail_allocation(source_df, source_pitch, final_products)


In [214]:
source_left


2490

In [208]:
combination

[[7000, 640, 640, 640, 640, 640, 640, 640, 640, 640, 640, 400],
 [7000,
  640,
  640,
  400,
  400,
  400,
  400,
  400,
  400,
  400,
  400,
  400,
  400,
  400,
  400,
  400],
 [3000, 400, 400, 400, 400, 400, 400, 280],
 [3000, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280],
 [3000, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280],
 [3000, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280],
 [3000, 280, 280, 280]]

In [209]:
eff

0.8689655172413793